In [2]:
import numpy as np
import pickle
import os

In [4]:
D = 5
K = 6
N = 10

In [5]:
ratings = [5,5,4,3,0]

In [10]:
W = {}

In [11]:
for k in range(K):
    W['w_'+str(k)] = np.random.randint(0,high = 5,size=(D,N) ,dtype='int16')

In [32]:
W2 = np.zeros((D,K,N),dtype='int16')

In [34]:
for k in range(K):
    for d in range(D): 
        W2[d][k] = W['w_'+str(k)][d]

In [36]:
W2.shape

(5, 6, 10)

In [147]:
class LRR_Model:
    global W2,ratings
    W = W2
    SCORE_SQUARE = False
    
    K = None # num of aspects
    D = None # num of documents
    N = None # num of words
    
    T = 1000 # max iteration
    
    mu_T = [0 for _ in range(T)]
    sigma_T = [0 for _ in range(T)]
    sigma_inv_T = [0 for _ in range(T)]
    delta_T = [0 for _ in range(T)]
    beta_T = [0 for _ in range(T)]
    
    aspect_weight_T = [0 for _ in range(T)] # alpha
    aspect_rating_T = [0 for _ in range(T)] # S
    
    def __init__(self,k,d,n):
        self.K = k
        self.D = d
        self.N = n
        
        mu_T[0] = np.array(np.random.rand(self.K) * 2.0 - np.ones((self.K),dtype='float32'))
        sigma_T[0] = np.array(np.identity(self.K,dtype='float32'))
        sigma_inv_T[0] = np.linalg.inv(sigma_T[0])
        
        beta_T[0] = (2*np.random.rand(self.K, self.N)-1.0)/10
        delta_T[0] = 1.0
        
    
#     def calc_covariance(self,vct):
#         '''
#         :param vct:
#         :return:

#         double sum = 0, s;
# 		for(int i=0; i<m_k; i++){
# 			s = 0;
# 			for(int j=0; j<m_k; j++)
# 				s += vct[j] * m_sigma_inv[j][i];
# 			sum += s * vct[i];
# 		}
# 		return sum;

#         '''

#     def calc_det(self):
#         return np.linalg.det(self.sigma)

#     def calc_sigma_inv(self,scale):
#         self.sigma_inv = np.linalg.inv(self.sigma) * scale

    def save(self,file_name:str):
        print(os.path.join(os.getcwd(), os.path.join('model', file_name)))
        with open(os.path.join(os.getcwd(), os.path.join('model', file_name)), 'wb') as fp:
            pickle.dump(self,fp)

    @classmethod
    def load(cls,file_name:str):
        try:
            with open(os.path.join(os.getcwd(),os.path.join('model',file_name)),'rb') as fp:
                return pickle.load(fp)
        except Exception as e:
            print(e)
    
    def get_aspect_rating(self,beta,d,t):
        if self.SCORE_SQUARE==True:    
            self.aspect_rating_T[t] = 0.5*np.sum(np.multiply(beta,self.W[d]),axis=1)#to avoid negative rating 
        else:
            self.aspect_rating_T[t] = np.exp(np.sum(np.multiply(beta,self.W[d]),axis=1))#to avoid negative rating 
    
    def infer_aspect_weight():
        pass
    
    def e_step():
        #infer_aspect_weight()
        pass
    def m_step():
        pass
    
    def em_est(converge:float):
#         e_step()
#         m_step()
        pass

In [148]:
model = LRR_Model(6,5,10)

In [149]:
model.save('test.model')

C:\Users\Chankoo\Desktop\GitHub\BOAZ-projects\airbnb-NLP\model\test.model


In [150]:
model = LRR_Model.load('test.model')

In [151]:
model.get_aspect_rating(beta_0,0,0)

In [153]:
model.aspect_rating_T[0]

array([0.17475506, 0.33031899, 0.57029433, 0.18522655, 0.75022365,
       0.01186862])